In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [4]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='5um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2-0.2
    resonator_length_2.append(a)
print(resonator_length_2)


[7.253254683870284, 7.024788339841421, 6.80834654020913]


# Flipchip

In [6]:
#V10부터 pad gap 바뀜
#V11 pad gap 바뀌면서 coupler&pad사이 거리 바뀐거 compensation -> big g
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            pad_gap='20um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+75}um",
                                              pad_width="110um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [13]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 10:51PM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:51PM [load_ansys_project]: 	Opened Ansys App
INFO 10:51PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 10:51PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 10:51PM [connect_design]: No active design found (or error getting active design).
INFO 10:51PM [connect]: 	 Connected to project "Project20". No design detected


True

In [15]:
hfss.activate_ansys_design("flip5um_V1", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

10:51PM 29s WARNING [activate_ansys_design]: The design_name=flip5um_V1 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 10:51PM [connect_design]: 	Opened active design
	Design:    flip5um_V1 [Solution type: Eigenmode]
WARNING 10:51PM [connect_setup]: 	No design setup detected.
WARNING 10:51PM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:51PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [16]:
hfss.close()

True

# LOM Analysis

In [13]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [14]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='4.58',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '4.58',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [16]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 09:12AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwrxsjm0v.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4580000000, Maxwell, 1, False
INFO 09:12AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp9zkecwvp.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4580000000, Maxwell, 1, False
INFO 09:12AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpb2x1omho.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4580000000, Maxwell, 2, False
INFO 09:12AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpz5u2uc62.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4580000000, Maxwell, 3, False
INFO 09:12AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpyuu08vn7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4580000000, Maxwell, 4, False
INFO 09:12

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,470.96,-145.0900,-67.340,-197.2900
pad_bot_Q0,-145.09,180.1500,-32.935,-1.0498
pad_top_Q0,-67.34,-32.9350,179.610,-78.2560
quad_coupler_c0,-197.29,-1.0498,-78.256,277.9600


In [17]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.34
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.432978 [GHz]
EC 179.572283 [MHz]
EJ 14.854145 [GHz]
alpha -198.831859 [MHz]
dispersion 0.045045 [KHz]
Lq 10.995564 [nH]
Cq 107.868697 [fF]
T1 18.776717 [us]

**Coupling Properties**

tCqbus1 -38.677014 [fF]
gbus1_in_MHz -414.821506 [MHz]
χ_bus1 -8.081137 [MHz]
1/T1bus1 8476.186060 [Hz]
T1bus1 18.776717 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,4.942257,225.625621,14.854145,-253.578206,0.671125,[-165.55178959750745],[-2.2805589210090433],2.280559,165.551790
2,4.848833,216.741191,14.854145,-242.910547,0.426867,[-459.05558526530854],[-15.7343019918122],15.734302,459.055585
3,4.472276,182.920436,14.854145,-202.765789,0.056729,[-480.99877351385334],[-11.339517811664672],11.339518,480.998774
4,4.468616,182.607165,14.854145,-202.397399,0.055533,[-488.0918010631939],[-11.629948969138589],11.629949,488.091801
5,4.452333,181.217049,14.854145,-200.763472,0.050489,[-455.3107488647053],[-9.942480274532063],9.942480,455.310749
6,4.438868,180.071938,14.854145,-199.418467,0.046641,[-428.15765058374643],[-8.664297060194471],8.664297,428.157651
7,4.422942,178.722673,14.854145,-197.834768,0.04244,[-415.72492265690187],[-8.028534577016645],8.028535,415.724923
8,4.433327,179.601863,14.854145,-198.866582,0.045138,[-416.3643567214353],[-8.144444025960121],8.144444,416.364357
9,4.432978,179.572283,14.854145,-198.831859,0.045045,[-414.8215059452446],[-8.081137061587373],8.081137,414.821506


In [18]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
